In [24]:
import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
import json
import os
from dotenv import load_dotenv, dotenv_value
llm = ChatOpenAI(temperature=0,model="gpt-4o-mini",api_key=os.getenv("OPENAI_API_KEY"))

load_dotenv()

In [26]:
### Load Document ###
# Update to handle PDF document loading
pdf_path = "Dental_Assitant_Book.pdf"  # Replace with your PDF file path
loader = PyPDFLoader(pdf_path,extract_images=True)
docs = loader.load()

### Split Document ###
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

C:\Users\admin\anaconda3\envs\GenAIWorkSpace\lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:125: UserWarning: Unknown PDF Filter!
  warnings.warn("Unknown PDF Filter!")


In [27]:
### Construct Retriever ###
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY")))
retriever = vectorstore.as_retriever()

In [28]:
### Contextualize Question ###
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

### Answer Question ###
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

### Statefully Manage Chat History ###
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [40]:
while True:
    _input = input("User::")
    if _input == 'exit()':
        print("Thank you for using me you can come back to continue..", end='\n')
        break
    else:
        response = conversational_rag_chain.invoke(
            {"input": _input},
            config={
                "configurable": {"session_id": "4565"}
            },
        )["answer"]
        print(response, end='\n')

User:: What is rubber DAM


A rubber dam is a dental tool made of a thin, flexible sheet of latex or latex-free material that is used to isolate specific teeth during dental procedures. It serves several important functions:

- **Isolation**: The rubber dam keeps the treatment area dry and free from saliva, blood, and other contaminants, which is essential for successful procedures like fillings, root canals, and crowns.
- **Visibility**: By isolating the teeth, it enhances visibility for the dentist, allowing for more precise work.
- **Infection Control**: It acts as a barrier to reduce the risk of contamination from bacteria and debris.
- **Patient Comfort**: The rubber dam minimizes the risk of water and materials entering the patient's mouth, helping to reduce anxiety during treatment.

Overall, the rubber dam is a valuable tool in modern dentistry that improves the quality and safety of dental care.


User:: Elaborate all the points


Certainly! Here’s a more detailed explanation of each point regarding rubber dams in dentistry:

### 1. **Isolation**
- **Function**: The primary purpose of a rubber dam is to isolate the specific teeth being treated from the rest of the oral cavity. This isolation is crucial for maintaining a clean and dry working environment.
- **Importance**: Many dental procedures require a dry field to ensure that materials (like dental adhesives or fillings) bond properly. Moisture can compromise the effectiveness of these materials, leading to treatment failure.

### 2. **Visibility**
- **Enhanced Access**: By isolating the treatment area, the rubber dam allows the dentist to have a clear view of the tooth or teeth being worked on. This is particularly important in complex procedures where precision is key.
- **Reduced Distractions**: The rubber dam also helps to retract the lips, tongue, and cheeks, minimizing distractions and allowing the dentist to focus solely on the procedure without interf

KeyboardInterrupt: Interrupted by user